In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import re

In [2]:
da = pd.read_excel('다나와 상품.xlsx')
da = da.drop_duplicates(subset=['name.1'])

co = pd.read_excel('커뮤니티 상품 DB.xlsx')
co = co.drop_duplicates(subset=['title'])

def preprocess_text(text):
    if isinstance(text, str):
        return re.sub(r'[^\s\d\w]', '', text)
    return ''

da['name.1'] = da['name.1'].apply(preprocess_text)
co['title'] = co['title'].apply(preprocess_text)

da['name.1'] = da['name.1'].fillna('').astype(str)
co['title'] = co['title'].fillna('').astype(str)

In [3]:
texts = da['name.1'].tolist() + co['title'].tolist()

In [4]:
# TF-IDF 벡터화
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(texts)


In [5]:
from sklearn.cluster import DBSCAN

# 최적의 eps와 min_samples 설정
optimal_eps = 0.5  # K-거리 그래프에서 결정된 값
min_samples = 5  # 일반적으로 데이터의 차원 수 + 1 이상

# DBSCAN 클러스터링
dbscan = DBSCAN(eps=optimal_eps, min_samples=min_samples)
clusters = dbscan.fit_predict(X)

# 클러스터링 결과 평가
num_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
print(f'클러스터 수: {num_clusters}')




클러스터 수: 150


In [9]:
# 실루엣 스코어
if num_clusters > 1:
     score = silhouette_score(X, clusters)
     print(f'실루엣 스코어: {score}')
else:
     print('측정되지 않음')

실루엣 스코어: -0.2895153177238738


In [6]:
clustered_texts = pd.DataFrame({'text': texts, 'cluster': clusters})
for cluster_num in range(num_clusters):
    print(f"\nCluster {cluster_num}:")
    cluster_texts = clustered_texts[clustered_texts['cluster'] == cluster_num]['text'].values
    for text in cluster_texts:  
        print(text)
    
    



Cluster 0:
황성주 국산콩 두유 검은콩 무가당
황성주 박사의 국산콩 두유 검은콩
황성주 박사의 국산콩 두유 검은콩 고구마
황성주 박사의 국산콩 두유 검은콩 고칼슘
황성주 박사의 국산콩 두유 검은콩 단호박
황성주 박사의 국산콩 두유 검은콩 무가당
황성주 박사의 국산콩 두유 검은콩 밤
황성주 박사의 국산콩 두유 무가당

Cluster 1:
상하목장 유기농 멸균 우유
상하목장 유기농 우유 125ml 멸균
상하목장 유기농 우유 190ml 멸균
상하목장 유기농 우유 200ml 멸균
상하목장 유기농 우유 900ml 멸균

Cluster 2:
소화가 잘되는 우유 바나나 락토프리 우유  멸균
소화가 잘되는 우유 오리지널 락토프리
소화가 잘되는 우유 오리지널 락토프리 우유  멸균
소화가 잘되는 우유 오리지널 락토프리 우유 190ml 멸균
소화가 잘되는 우유 저지방 락토프리
소화가 잘되는 우유 저지방 락토프리 우유  멸균
소화가 잘되는 우유 저지방 락토프리 우유 190ml 멸균
소화가 잘되는 우유 초콜릿 락토프리 우유  멸균

Cluster 3:
X
F
잣

L
츄 2
첨
格 격
 뭐

 


펑






  




 

   


ฏ

1





뻥


o

Cluster 4:
엘리트북 645 G10 80D10PA
엘리트북 645 G10 80D10PA 16GB램
엘리트북 645 G10 80D10PA 32GB램
엘리트북 645 G10 80D10PA 64GB램
엘리트북 645 G10 80D10PA WIN11

Cluster 5:
BlackShark V2
BlackShark V2 Pro
BlackShark V2 Pro 2023
BlackShark V2 X
BlackShark V2 X USB
Blackshark V2

Cluster 6:
칼슘 마그네슘 아연 비타민D 90정
칼슘 마그네슘 아연 비타민D 맥스 90정
프리미엄 칼슘 마그네슘 아연 비타민D 90정
칼슘 마그네슘 아연 플러스 비타민D 90정
칼슘 마그네슘 아연 앤 비타민D 밸런스 90정

Cluster 7:
정

In [8]:
with open('클러스터 결과.txt', 'w', encoding='utf-8') as file:
    for cluster_num in range(num_clusters):
        file.write(f"\nCluster {cluster_num}:\n")
        cluster_texts = clustered_texts[clustered_texts['cluster'] == cluster_num]['text'].values
        for text in cluster_texts:
            file.write(text + '\n')